In [1]:
import gc
import pickle
import itertools
from pathlib import Path
from multiprocessing import Pool

import cv2
import jpeg4py
import torch
import numpy as np
import pandas as pd
import albumentations as albu
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from effdet import DetBenchPredict
from ensemble_boxes import (nms, soft_nms, non_maximum_weighted,
                            weighted_boxes_fusion, weighted_boxes_fusion_3d)

from src.model.detector import get_net
from src.data.clip_dataset import ClipDataset
from src.data.loaders import get_valid_dataloaders, get_dataloaders, get_image_dataloaders
from src.data.utils import read_video, write_video
from src.metric import precision_calc, comp_metric, iou
from src.postprocessing import filter_by_frame, threshold_preds, postprocessing_video
from src.visualization import draw_hits_video
from config import data as data_cfg, train as train_cfg, model as model_cfg
from train import Fitter

In [1]:
# it = get_valid_dataloaders(data_cfg, train_cfg)
# mid_frame = np.where(np.array(data_cfg.frames_neighbors) == 0)[0][0]
# device = torch.device('cuda:1')
# net = get_net(model_cfg, num_classes=2, mid_frame=mid_frame).to(device)
# fitter = Fitter(model=net, device=device)

# fitter.model.eval()
# inference_model = DetBenchPredict(fitter.model.model).to(device)

In [3]:
# gt_all, preds_all, scores_all = [], [], []

# for loader in tqdm(it, total=24):
#     gt, video_preds, video_scores = fitter.predict_single_video(inference_model, loader)
#     gt_all += [gt]
#     preds_all += [video_preds]
#     scores_all += [video_scores]

# data = {
#     'gt_all': gt_all,
#     'preds_all': preds_all,
#     'scores_all': scores_all
# }
# with open('data.pickle', 'wb') as f:
#     pickle.dump(data, f)

In [4]:
# import pickle
# with open('data.pickle', 'rb') as f:
#     data = pickle.load(f)
    
# gt_all = data['gt_all']
# preds_all = data['preds_all']
# scores_all = data['scores_all']

In [31]:
def get_scores(params):
    global_thr, nms_iou_thr, same_helmet_iou_thr, look_future_n_frames, track_max_len = params
    preds = [
    postprocessing_video(
        preds, scores, 
        global_thr=global_thr,
        nms_iou_thr=nms_iou_thr,
        same_helmet_iou_thr=same_helmet_iou_thr,
        look_future_n_frames=look_future_n_frames,
        track_max_len=track_max_len,
    )[0] for preds, scores in zip(preds_all, scores_all)]

    precision, recall, f1, _ = comp_metric(preds, gt_all)
    
    return precision, recall, f1, params



it_params = itertools.product(
    np.arange(0.2, 0.3, 0.02),
    [0.3, 0.35],
    [0.3, 0.35],
    [8, 14, 20],
    range(8, 15, 2),
)

with Pool(16) as p:
    result = p.map(get_scores, it_params)
result = sorted(result, key=lambda x: -x[2])

In [37]:
boxes_frame = np.array([[0.82734375, 0.82638889, 0.9421875, 0.87638889]])
scores_frame = np.array([0.05685425])

boxes_nms, scores_nms, _ = nms(
    [boxes_frame],
    [scores_frame],
    [np.ones_like(scores_frame)],
    iou_thr=0.3,
)

In [ ]:
# gt

In [ ]:
# preds

In [ ]:
# scores

In [ ]:
video = draw_hits_video(loader.dataset.video_path, preds, scores, out_path='out.mp4')

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(video[0])